In [1]:
! pip install -U accelerate
! pip install -U transformers
! pip install kss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoTokenizer, PreTrainedTokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, AutoModelForCausalLM, DataCollatorForLanguageModeling
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader

Mounted at /content/drive


In [ ]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset, DataLoader
import kss  # 한국어 문장 분리 라이브러리
from sklearn.model_selection import train_test_split

# 토크나이저 및 모델 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

# 동화 파일 읽기 및 전처리
def read_tales(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        tales = f.read().split('\n\n\n')
    formatted_tales = []
    for tale in tales:
        # kss로 문장 분리
        sentences = kss.split_sentences(tale)
        # 각 문장 앞뒤에 </s> 추가
        formatted_sentences = ['</s> ' + sentence.strip() + ' </s>' for sentence in sentences if sentence.strip()]
        formatted_tale = ' '.join(formatted_sentences)
        formatted_tales.append(formatted_tale)
    return formatted_tales

class TaleDataset(Dataset):
    def __init__(self, tales, tokenizer, max_length=1024):
        self.tales = tales
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.tales)

    def __getitem__(self, idx):
        tale = self.tales[idx]
        tokens = self.tokenizer(tale, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        input_ids = tokens['input_ids'].squeeze(0)
        attention_mask = tokens['attention_mask'].squeeze(0)
        labels = input_ids.clone()  # labels 추가
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels  # labels 반환
        }

# 긴 동화를 슬라이딩 윈도우로 분할
def split_long_tales(tale, max_length=512, stride=256):
    tokens = tokenizer(tale, return_tensors='pt')['input_ids'].squeeze(0)
    split_tales = []
    for i in range(0, len(tokens), stride):
        split_tale = tokens[i:i+max_length]
        if len(split_tale) < max_length:
            split_tale = torch.cat([split_tale, torch.tensor([tokenizer.pad_token_id] * (max_length - len(split_tale)))])
        split_tales.append(split_tale)
    return split_tales

# 동화 파일 경로
file_path = 'path/to/your/tales.txt'

# 동화 읽기 및 전처리
tales = read_tales(file_path)

# 긴 동화 슬라이딩 윈도우 적용 (선택적)
tales = [split_long_tales(tale) for tale in tales]
tales = [item for sublist in tales for item in sublist]  # 리스트 평탄화

# 학습 및 검증 데이터 분할
train_tales, val_tales = train_test_split(tales, test_size=0.1, random_state=42)

# 데이터셋 생성
train_dataset = TaleDataset(train_tales, tokenizer)
val_dataset = TaleDataset(val_tales, tokenizer)

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
# 모델 학습 설정
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
)

# Data Collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

# 모델 학습
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,No log


KeyError: 'eval_loss'

In [ ]:
import torch
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# 모델과 토크나이저 불러오기
model_dir = '/content/drive/MyDrive/Tale/models/final_model'
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained(model_dir)

# 생성할 문장 설정
prompt_text = "옛날에 고래 백경이가 살았어요."

# 입력 문장 토크나이징 및 특수 토큰 추가
inputs = tokenizer(prompt_text, return_tensors="pt")

# 문장 생성
max_length = 50  # 최대 생성 길이
temperature = 0.7  # 0.7로 설정하여 다양성 증가
top_p = 0.9  # 누적 확률 상위 90%의 토큰만 고려
top_k = 50  # 확률 상위 50개 토큰만 고려

output = model.generate(inputs['input_ids'], max_length=max_length, num_return_sequences=1,
                        do_sample=True, temperature=temperature, top_p=top_p, top_k=top_k,
                        no_repeat_ngram_size=2)

# 생성된 문장 디코딩
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Sentence:", generated_text)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Generated Sentence: 옛날에 고래 백경이가 살았어요.요.요요 요 요요요
요
요료료요요로요로요요인요인 요인 요인요인요소요소요인요요가요가 요가 요가요가요요,요,요,
요,
요,가,가,나,나,나나나,


In [1]:
from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoTokenizer, PreTrainedTokenizerFast, Trainer, TrainingArguments, AutoModelForCausalLM, DataCollatorForLanguageModeling
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import numpy as np
from kss import split_sentences  # kss의 split_sentences 함수를 사용하기 위해 import

# 동화 파일 경로
txt_file_path = "/content/drive/MyDrive/Tale/tale.txt"

# 토크나이저 및 모델 초기화
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
model.config.pad_token_id = model.config.eos_token_id

# 동화 파일 읽기 및 전처리 함수
def read_tales(file_path, tokenizer):
    with open(file_path, 'r', encoding='utf-8') as f:
        tales = f.read().split('\n\n\n')

    formatted_tales = []
    for tale in tales:
        # kss로 문장 분리
        sentences = split_sentences(tale)
        # 각 문장 앞뒤에 </s> 추가하여 포맷팅
        formatted_sentences = []
        for sentence in sentences:
            if sentence.strip():
                formatted_sentence = f"</s> {sentence.strip()} </s>"
                formatted_sentences.append(formatted_sentence)

        # 동화 전체를 하나의 문자열로 결합
        formatted_tale = ' '.join(formatted_sentences)
        formatted_tales.append(formatted_tale)

    # 토크나이저를 사용하여 토큰화
    tokenized_tales = [tokenizer(tale, truncation=True, max_length=1024) for tale in formatted_tales]

    return tokenized_tales

# 동화 데이터셋 클래스
class FairyTaleDataset(Dataset):
    def __init__(self, tokenized_tales):
        self.tokenized_tales = tokenized_tales

    def __len__(self):
        return len(self.tokenized_tales)

    def __getitem__(self, idx):
        return self.tokenized_tales[idx]

# 데이터셋 및 데이터 콜레이터 설정
tokenized_tales = read_tales(txt_file_path, tokenizer)
train_size = int(0.8 * len(tokenized_tales))
train_dataset = FairyTaleDataset(tokenized_tales[:train_size])
val_dataset = FairyTaleDataset(tokenized_tales[train_size:])
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [2]:
# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Tale/kogpt2_fairy_tales",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    prediction_loss_only=True,
    logging_steps=100,
)

# Perplexity 계산 함수
def compute_perplexity(model, dataset, data_collator, device):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=8, collate_fn=data_collator, shuffle=False)
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            non_padded_tokens = torch.sum(attention_mask)
            total_loss += loss.item() * non_padded_tokens.item()
            total_tokens += non_padded_tokens.item()

    avg_loss = total_loss / total_tokens
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    return perplexity

# Trainer 설정 및 학습
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Fine-tuning 전 Perplexity 계산
pre_train_perplexity = compute_perplexity(model, val_dataset, data_collator, device)
print(f"Fine-tuning 전 Perplexity: {pre_train_perplexity:.2f}")

# 모델 학습
trainer.train()

# Fine-tuning 후 Perplexity 계산
post_train_perplexity = compute_perplexity(model, val_dataset, data_collator, device)
print(f"Fine-tuning 후 Perplexity: {post_train_perplexity:.2f}")

# 최종 모델 저장
trainer.save_model("/content/drive/MyDrive/Tale/fine_tuned_kogpt2_fairy_tales")


Fine-tuning 전 Perplexity: 64.33


Epoch,Training Loss,Validation Loss
1,No log,2.942242
2,3.161800,2.994435
3,3.161800,3.042810
4,2.436400,3.129135
5,1.884800,3.228823
6,1.884800,3.309244
7,1.429200,3.372385
8,1.429200,3.431200
9,1.096500,3.532061
10,0.791100,3.584879


Fine-tuning 후 Perplexity: 137.62


In [4]:
! pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=3fcea42d40f70a9f60b9255bdf225c9c47ceb8e5b57659e84a671bf26f8d592f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [8]:
import torch
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.utils.data import Dataset, DataLoader
import kss  # 한국어 문장 분리 라이브러리
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
import numpy as np

# 토크나이저 및 모델 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                                                    pad_token='<pad>', mask_token='<mask>')
base_model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
fine_tuned_model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/Tale/fine_tuned_kogpt2_fairy_tales")

# Perplexity 계산 함수
def compute_perplexity(model, dataset, data_collator, device):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=8, collate_fn=data_collator, shuffle=False)
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            non_padded_tokens = torch.sum(attention_mask)
            total_loss += loss.item() * non_padded_tokens.item()
            total_tokens += non_padded_tokens.item()

    avg_loss = total_loss / total_tokens
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    return perplexity

# BLEU 점수 계산 함수
def compute_bleu(model, dataset, tokenizer, device):
    model.eval()
    bleu_scores = []

    with torch.no_grad():
        for example in dataset:
            input_ids = example['input_ids'].unsqueeze(0).to(device)
            attention_mask = example['attention_mask'].unsqueeze(0).to(device)

            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1024, num_return_sequences=1)
            decoded_preds = tokenizer.decode(outputs[0], skip_special_tokens=True)
            decoded_labels = tokenizer.decode(example['input_ids'], skip_special_tokens=True)

            reference = [decoded_labels.split()]
            candidate = decoded_preds.split()
            bleu_score = sentence_bleu(reference, candidate)
            bleu_scores.append(bleu_score)

    avg_bleu = np.mean(bleu_scores)
    return avg_bleu

# ROUGE 점수 계산 함수
def compute_rouge(model, dataset, tokenizer, device):
    model.eval()
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    with torch.no_grad():
        for example in dataset:
            input_ids = example['input_ids'].unsqueeze(0).to(device)
            attention_mask = example['attention_mask'].unsqueeze(0).to(device)

            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=1024, num_return_sequences=1)
            decoded_preds = tokenizer.decode(outputs[0], skip_special_tokens=True)
            decoded_labels = tokenizer.decode(example['input_ids'], skip_special_tokens=True)

            rouge_scores = rouge_scorer_obj.score(decoded_labels, decoded_preds)
            rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
            rouge2_scores.append(rouge_scores['rouge2'].fmeasure)
            rougeL_scores.append(rouge_scores['rougeL'].fmeasure)

    avg_rouge1 = np.mean(rouge1_scores)
    avg_rouge2 = np.mean(rouge2_scores)
    avg_rougeL = np.mean(rougeL_scores)
    return avg_rouge1, avg_rouge2, avg_rougeL

# 데이터 준비
file_path = '/content/drive/MyDrive/Tale/tale.txt'
tales = read_tales(file_path, tokenizer)
train_tales, val_tales = train_test_split(tales, test_size=0.1, random_state=42)
train_dataset = FairyTaleDataset(train_tales, tokenizer)
val_dataset = FairyTaleDataset(val_tales, tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 기기 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model.to(device)
fine_tuned_model.to(device)

# 평가
print("기존 모델 평가:")
base_perplexity = compute_perplexity(base_model, val_dataset, data_collator, device)
base_bleu = compute_bleu(base_model, val_dataset, tokenizer, device)
base_rouge1, base_rouge2, base_rougeL = compute_rouge(base_model, val_dataset, tokenizer, device)
print(f"기존 모델 Perplexity: {base_perplexity:.2f}")
print(f"기존 모델 BLEU: {base_bleu:.2f}")
print(f"기존 모델 ROUGE-1: {base_rouge1:.2f}, ROUGE-2: {base_rouge2:.2f}, ROUGE-L: {base_rougeL:.2f}")

print("\nFine-tuning된 모델 평가:")
fine_tuned_perplexity = compute_perplexity(fine_tuned_model, val_dataset, data_collator, device)
fine_tuned_bleu = compute_bleu(fine_tuned_model, val_dataset, tokenizer, device)
fine_tuned_rouge1, fine_tuned_rouge2, fine_tuned_rougeL = compute_rouge(fine_tuned_model, val_dataset, tokenizer, device)
print(f"Fine-tuning된 모델 Perplexity: {fine_tuned_perplexity:.2f}")
print(f"Fine-tuning된 모델 BLEU: {fine_tuned_bleu:.2f}")
print(f"Fine-tuning된 모델 ROUGE-1: {fine_tuned_rouge1:.2f}, ROUGE-2: {fine_tuned_rouge2:.2f}, ROUGE-L: {fine_tuned_rougeL:.2f}")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


TypeError: FairyTaleDataset.__init__() takes 2 positional arguments but 3 were given

In [9]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch

# Fine-tuning된 모델 경로
model_path = "/content/drive/MyDrive/Tale/fine_tuned_kogpt2_fairy_tales"

# 모델과 토크나이저 로드
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = PreTrainedTokenizerFast.from_pretrained('skt/kogpt2-base-v2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_text(prompt, max_length=100, num_return_sequences=1, temperature=0.7, top_k=50, top_p=0.95):
    # 입력 프롬프트를 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # 텍스트 생성
    outputs = model.generate(
        input_ids=inputs.input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,  # 반복 방지
        do_sample=True,  # 샘플링 여부
        top_k=top_k,  # 샘플링할 토큰의 개수
        top_p=top_p,  # 누적 확률을 통한 샘플링
        temperature=temperature,  # 샘플링의 다양성 조정
        eos_token_id=tokenizer.eos_token_id  # 종료 토큰
    )

    # 토큰을 텍스트로 변환
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts

# 프롬프트 설정
prompt = "고래 백경이는 바다 마을에 사는 마음씨가 착한 고래랍니다. 어느 날 백경이는 육지로 모험을 떠났어요. 그러다 마음씨 착한 토끼 토순이를 만났죠."

# 텍스트 생성
generated_text = generate_text(prompt, max_length=200, num_return_sequences=1)[0]
print("Generated Text:\n", generated_text)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Generated Text:
 고래 백경이는 바다 마을에 사는 마음씨가 착한 고래랍니다. 어느 날 백경이는 육지로 모험을 떠났어요. 그러다 마음씨 착한 토끼 토순이를 만났죠.  껑충껑충을 뛰어 돌고 난 백경은 그때, "백경아, 네가 정말 뭔가를 잃어버렸다고 믿기 어렵구나." 윙윙 맴돈다. 꾹 참고 할 수 있는 최대한 죽은 척을 하고 있는 백 경장은 곤욕을 당했다.  포유 강에 사는 용왕님께 제물로 어린 처녀를 바친다는 소식을 듣고 백 씨는 집으로 달려갔다. 라며 어린 여우가 말했다. 라면 맛 좋은 건 다 먹었지만, 바다에서 죽은 용왕님을 발견했을 때 마다 용왕님은 다른 사람들에게 용왕님의 상태를 본 후 말했었다.   보고서 보니 자신이 용왕님이 잃어버린 물건을 찾을 수 있다는 희망때문에 덜컥 약속을 해버린 자신을 원망했다...  묻혀 있는 용왕품을 찾을 때까지만 기다려주렴." 라며, 여우는 용왕에게 간곡히 부탁했다.. 라면서
